## CV16 - 불안한 시선 이펙트 추가하기
mkdir -p ~/aiffel/coarse_to_fine/data  
cd ~/aiffel/coarse_to_fine  
wget https://aiffelstaticprd.blob.core.windows.net/media/documents/coarse_to_fine_pjt.zip  
unzip coarse_to_fine_pjt.zip  
### 위치 측정을 위한 라벨링 툴 만들기
#### OpenCV 사용
품질이 좋지 못한 데이터에 잘못 라벨링 되었을 경우 올바른 위치에 눈동자를 지정하여 fine라벨로 만들어야 한다.  
이때 눈동자 위치를 선택할 수 있는 도구가 있어야 한다.  
openCV에서는 마우스 이벤트를 callback함수 형태로 지원한다.  
callback함수 : https://satisfactoryplace.tistory.com/18  
openCV에서 지원하는 마우스 이벤트 형태 : https://opencv-python.readthedocs.io/en/latest/doc/04.drawWithMouse/drawWithMouse.html  
#### 툴 만들기
keypoint_using_mouse.py을 이용하여 눈동자에 마우스 클릭하여 s을 입력하여 좌표를 저장한다.  
#### 데이터를 모아보자
라벨링할 초기 데이터 수집, 직접 촬영하거나 수집해도 되지만 공개된 데이터를 이용하자.  
눈동자 위치를 필요로 하기 때문에  
1. 눈이 크롭되어 있고 눈동자 위치를 라벨로 가지고 있는 데이터  
2. 얼굴 랜드마크를 가지고 있는 데이터  
3. 얼굴 이미지를 가지고 있는 데이터  
순으로 찾으면 좋겠다.  
1번에 해당하는 데이터셋 - BioID : https://www.bioid.com/facedb/
1521장은 너무 작다. 얼굴 랜드마크가 제공되는 데이터를 찾아보자. 랜드마크가 제공되니깐 눈 부분 크롭 가능  
dlib은 얼굴 랜드마크를 사용하는데 어떤 학습 데이터셋을 사용했을 것이다. 그것을 찾아보자.  
dlib face landmark dataset 구글 검색  
http://dlib.net/face_landmark_detection.py.html 찾았다. iBUG 300-W라는 데이터셋으로 학습했다고...  
운이 좋군. 하지만 실무에서는 대부분 관련 도메인 데이터셋이 없는 경우가 많다고 한다.  
이를 경우 3번 경우처럼 얼굴 이미지 데이터셋에서 랜드마크를 추출하고 눈 부분을 크롭한 후 라벨링해야 한다.  
LFW데이터셋은 안면 인식과 관련된 데이터셋으로 얼굴 이미지만 있다.  

wget http://vis-www.cs.umass.edu/lfw/lfw.tgz  
mv lfw.tgz ~/aiffel/coarse_to_fine/data  
cd ~/aiffel/coarse_to_fine/data && tar -xvzf lfw.tgz  
### Mean-shift를 이용한 눈동자 검출 방법
#### 이론
눈동자는 어떻게 찾을 수 있을까? 눈동자는 가운데 어두운 색을 가지고 있다. 색 반전 후 밝은 색, 최대값을 찾는 방법이 있다.  
하지만 머리카락이 내려와 겹쳐있을 경우 눈의 가장자리 혹은 다른 곳을 찾게 될 수 있다.  
2차원 블러 특성 이미지에서 눈동자에 2차원의 정규분포로 나타나는 영역이 보인다.  
1차원 누적 그래프로 보면 x축으로 2개의 봉우리가 보인다. 가장자리부터 누적된 값을 확인하면 최대값인 곳에 도달할 것이다.  
2D에서 최고점을 찾는 방법  
1. 이미지 중심점을 초기값으로 설정: 눈의 중심에 눈동자가 있을 확률이 높기 때문에
2. 중심점을 기준으로 작은 box을 설정: box의 크기는 상황에 따라 적절하게 설정
3. box내부의 픽셀값을 이용해서 무게중심 찾기: pixel intensity를 weight로 사용 가능  
4. 찾은 무게중심을 기준을 box의 중심으로 설정: 박스가 이동
5. 이동한 지점에서 위 과정을 반복
6. 수렴할 때까지 수 번을 반복하면 눈동자 찾을 수 있다.  
머신러닝에 Mean Shift라는 알고리즘이 이런 이론이다.  
탐색반경 내 데이터 포인트의 평균을 구하고 그 평균 위치로 이동, 이 과정을 반복하면서 데이터 분포의 중심으로 이동  
영상추적 - mean shift 추적 : https://darkpgmr.tistory.com/64  

#### 실습
cd ~/aiffel/coarse_to_fine && python eye_center_basic.py True  
True옵션은 매 스텝의 작동을 확인 가능, False나 생략하면 최종 결과만 확인 가능  
cd ~/aiffel/coarse_to_fine && cp eye_center_basic.py eye_center_meanshift.py  
eye_center_meanshift.py에서 findCenterPoint함수를 다음과 같이 수정한다.

In [ ]:
def findCenterPoint(gray_eye, str_direction='left'):
    if gray_eye is None:
        return [0, 0]
    filtered_eye = cv2.bilateralFilter(gray_eye, 7, 75, 75)
    filtered_eye = cv2.bilateralFilter(filtered_eye, 7, 75, 75)
    filtered_eye = cv2.bilateralFilter(filtered_eye, 7, 75, 75)

    # 2D images -> 1D signals
    row_sum = 255 - np.sum(filtered_eye, axis=0)//gray_eye.shape[0]
    col_sum = 255 - np.sum(filtered_eye, axis=1)//gray_eye.shape[1]

    # normalization & stabilization
    def vector_normalization(vector):
        vector = vector.astype(np.float32)
        vector = (vector-vector.min())/(vector.max()-vector.min()+1e-6)*255
        vector = vector.astype(np.uint8)
        vector = cv2.blur(vector, (5,1)).reshape((vector.shape[0],))
        vector = cv2.blur(vector, (5,1)).reshape((vector.shape[0],))            
        return vector
    row_sum = vector_normalization(row_sum)
    col_sum = vector_normalization(col_sum)

    def findOptimalCenter(gray_eye, vector, str_axis='x'):
        axis = 1 if str_axis == 'x' else 0
        center_from_start = np.argmax(vector)
        center_from_end = gray_eye.shape[axis]-1 - np.argmax(np.flip(vector,axis=0))
        return (center_from_end + center_from_start) // 2

        # x 축 center 를 찾는 알고리즘을 mean shift 로 대체합니다.
    # center_x = findOptimalCenter(gray_eye, row_sum, 'x')
    center_y = findOptimalCenter(gray_eye, col_sum, 'y')

        # 수정된 부분
    inv_eye = (255 - filtered_eye).astype(np.float32)
    inv_eye = (255*(inv_eye - inv_eye.min())/(inv_eye.max()-inv_eye.min())).astype(np.uint8)

    resized_inv_eye = cv2.resize(inv_eye, (inv_eye.shape[1]//3, inv_eye.shape[0]//3))
    init_point = np.unravel_index(np.argmax(resized_inv_eye),resized_inv_eye.shape)

    x_candidate = init_point[1]*3 + 1
    for idx in range(10):
        temp_sum = row_sum[x_candidate-2:x_candidate+3].sum()
        if temp_sum == 0:
            break
        normalized_row_sum_part = row_sum[x_candidate-2:x_candidate+3].astype(np.float32)//temp_sum
        moving_factor = normalized_row_sum_part[3:5].sum() - normalized_row_sum_part[0:2].sum()
        if moving_factor > 0.0:
            x_candidate += 1
        elif moving_factor < 0.0:
            x_candidate -= 1

    center_x = x_candidate

    if center_x >= gray_eye.shape[1]-2 or center_x <= 2:
        center_x = -1
    elif center_y >= gray_eye.shape[0]-1 or center_y <= 1:
        center_y = -1

    return [center_x, center_y]

첫번째, 눈 이미지를 low pass filter로 smoothing, bilateral filter사용  
두번째, 1차원 값을 누적하여 y축 기준의 최대값으로 중심점의 y축 좌표를 얻음.  
세번째, x축은 최대값 지점에서 mean shift를 적용하여 양 끝단에 수렴하는 예외는 처리한 후 x축 좌표를 얻음.  
### 키포인트 검출 딥러닝 모델 만들기
#### 데이터 확인
학습을 위해  대량의 눈동자 위치 라벨 필요. 10000개 이상의 데이터셋이 필요  
LFW데이터셋에 앞의 눈동자 검출 방법을 적용시켜 데이터셋을 생성한다.  
cd ~/aiffel/coarse_to_fine && python prepare_eye_dataset.py  
tensorflow hub에서 제공하는 pretrained image feature embedding으로 fine tuning  
ImageDataGenerator형식으로 데이터를 읽음, 라벨이 image형태로 저장되었음.  
train dataset 23712, val dataset 2638  
image_generator, label generator는 텐서플로우의 generator형식을 사용하였기에 출력 형식도 맞춘다.  
제너레이터 : https://tensorflow.blog/%ED%9A%8C%EC%98%A4%EB%A6%AC%EB%B0%94%EB%9E%8C%EC%9D%84-%ED%83%84-%ED%8C%8C%EC%9D%B4%EC%8D%AC/%EC%A0%9C%EB%84%88%EB%A0%88%EC%9D%B4%ED%84%B0/  
학습라벨에는 3개의 점이 lavel이미지에 표시되어 있다.  
눈의 왼쪽 끝을 1, 오른쪽 끝을 2, 중심을 3으로 지정하였고 np.where()함수로 이미지에서 좔표로 복원한다.  
아래 코드를 파일로 저장

In [1]:
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf
import numpy as np
from glob import glob

list_image = sorted(glob('./data/train/input/img/*.png'))
list_label = sorted(glob('./data/train/label/mask/*.png'))
print (len(list_image), len(list_label))

IMAGE_SHAPE = (80, 120)
data_root = './data/train/input'
label_root = './data/train/label'

image_generator = tf.keras.preprocessing.image.ImageDataGenerator()
label_generator = tf.keras.preprocessing.image.ImageDataGenerator()
image_data = image_generator.flow_from_directory(str(data_root), class_mode=None, target_size=IMAGE_SHAPE, batch_size=32)
label_data = label_generator.flow_from_directory(str(label_root), class_mode=None, target_size=IMAGE_SHAPE, batch_size=32)


def user_generation(train_generator, label_generator):
    h, w = train_generator.target_size
    for images, labels in zip(train_generator, label_generator):
        images /= 255.
        images = images[..., ::-1] # rgb to bgr

        list_point_labels = []
        for img, label in zip(images, labels):

            eye_ls = np.where(label==1) # leftside
            eye_rs = np.where(label==2) # rightside
            eye_center = np.where(label==3)

            lx, ly = [eye_ls[1].mean(), eye_ls[0].mean()]
            rx, ry = [eye_rs[1].mean(), eye_rs[0].mean()]
            cx, cy = [eye_center[1].mean(), eye_center[0].mean()]

            if len(eye_ls[0])==0 or len(eye_ls[1])==0:
                lx, ly = [0, 0]
            if len(eye_rs[0])==0 or len(eye_rs[1])==0:
                rx, ry = [w, h]
            if len(eye_center[0])==0 or len(eye_center[1])==0:
                cx, cy = [0, 0]

            np_point_label = np.array([lx/w,ly/h,rx/w,ry/h,cx/w,cy/h], dtype=np.float32)

            list_point_labels.append(np_point_label)
        np_point_labels = np.array(list_point_labels)
        yield (images, np_point_labels)
        
user_train_generator = user_generation(image_data, label_data)
for i in range(2):
    dd = next(user_train_generator)
    print (dd[0][0].shape, dd[1][0])

0 0


FileNotFoundError: [Errno 2] No such file or directory: './data/train/input'

#### 모델 설계
tensorflow Hub에서 resnet의 특성추출기 부분을 백본으로 사용  
데이터 제너레이터에서 출력을 6개(각 점당 좌표, 2 * 3)로 했기 때문에 num_classes는 6으로 설정  
position regression문제이므로 loss와 metric은 mse, mae로 설정한다.  

In [ ]:
''' tf hub feature_extractor '''
feature_extractor_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                            input_shape=(80,120,3))

image_batch = next(image_data)
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

num_classes = 6

feature_extractor_layer.trainable = False
model = tf.keras.Sequential([
    feature_extractor_layer,
    #layers.Dense(1024, activation='relu'),
    #layers.Dropout(0.5),
    layers.Dense(num_classes, activation='sigmoid'),
])

model.summary()

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='mse',
  metrics=['mae']
  )

def lr_step_decay(epoch):
    init_lr = 0.0005 #self.flag.initial_learning_rate
    lr_decay = 0.5 #self.flag.learning_rate_decay_factor
    epoch_per_decay = 2 #self.flag.epoch_per_decay
    lrate = init_lr * math.pow(lr_decay, math.floor((1+epoch)/epoch_per_decay))
    return lrate
    
steps_per_epoch = image_data.samples//image_data.batch_size
print (image_data.samples, image_data.batch_size, steps_per_epoch)
# 20160 32 630 -> 데이터를 batch_size 의 배수로 준비해 주세요.

learning_rate = LearningRateScheduler(lr_step_decay)

history = model.fit(user_train_generator, epochs=10,
                    steps_per_epoch=steps_per_epoch,
                    callbacks = [learning_rate]
                    )

#### 평가

In [ ]:
IMAGE_SHAPE = (80, 120)
val_data_root = './data/val/input'
val_label_root = './data/val/label'

image_generator_val = tf.keras.preprocessing.image.ImageDataGenerator()
label_generator_val = tf.keras.preprocessing.image.ImageDataGenerator()
image_data_val = image_generator.flow_from_directory(str(val_data_root),
                                                     class_mode=None,
                                                     target_size=IMAGE_SHAPE,
                                                     shuffle=False)
label_data_val = label_generator.flow_from_directory(str(val_label_root),
                                                     class_mode=None,
                                                     target_size=IMAGE_SHAPE,
                                                     shuffle=False)

user_val_generator = user_generation(image_data_val, label_data_val)
mse, mae = model.evaluate_generator(user_val_generator, image_data_val.n // 32)
print(mse, mae)

평균 에러가 0.013정도 나옴.  
120픽셀 기준으로 120 * 0.013= 1.56픽셀 정도 에러 발생  
이미지 출력

In [ ]:
# img test
img = cv2.imread('./data/val/input/img/eye_000010_l.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

입력 파이프에 맞게 이미지 리사이즈(120, 80)후 배치 차원을 추가하고 한 장만 출력하니깐 배치 size를 1로 한다.  
출력값은 좌측, 우측, 중앙좌표

In [ ]:
np_inputs = np.expand_dims(cv2.resize(img, (120, 80)), axis=0)
preds = model.predict(np_inputs/255., 1)

repred = preds.reshape((1, 3, 2))
repred[:,:,0] *= 120
repred[:,:,1] *= 80
print (repred)

이미지를 출력하려면 pt에 0.5를 곱해야 한다. 사용하는 데이터 크기가 60x40이므로 